In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

c:\source\repos\TextToHTMLApp\4990env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 4)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import numpy as np

In [4]:
import torch, torch.nn as nn
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [5]:
import pandas as pd

data_frame = pd.read_csv("../dataset/texts_and_labels.csv")
print(data_frame.head())

                                                Text   Label
0  New update in Valorant introduces a brand-new ...  gaming
1  Fortnite's latest season brings back fan-favor...  gaming
2  Review: The immersive world of Elden Ring offe...  gaming
3  How to dominate in League of Legends: Tips and...  gaming
4  Global economic summit outlines strategies for...    news


split dataset

In [6]:

genre_dict = {'gaming': 0, 'news': 1, 'education': 2, 'sports': 3}

data_frame['Label'] = data_frame['Label'].apply(lambda x: genre_dict[x])
data_frame


,Text,Label
0,New update in Valorant introduces a brand-new ...,0
1,Fortnite's latest season brings back fan-favor...,0
2,Review: The immersive world of Elden Ring offe...,0
3,How to dominate in League of Legends: Tips and...,0
4,Global economic summit outlines strategies for...,1
...,...,...
65,Competitive gaming leagues are becoming as pop...,0
66,Youth sports programs are essential for teachi...,3
67,Athletes are speaking out more on social issue...,3
68,This indie game's innovative gameplay and art ...,0


In [11]:
import datasets
raw_datasets = datasets.Dataset.from_pandas(data_frame)
raw_datasets

Dataset({
    features: ['Text', 'Label'],
    num_rows: 70
})

In [12]:
def tokenize_function(examples):
    return {'input_ids': tokenizer(examples["Text"], truncation=True)["input_ids"], 'labels': examples["Label"]}

In [17]:
from transformers import DataCollatorWithPadding
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)

tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)

tokenized_datasets

Map: 100%|██████████| 70/70 [00:00<00:00, 6363.68 examples/s]


DatasetDict({
    train: Dataset({
        features: ['Text', 'Label', 'input_ids', 'attention_mask'],
        num_rows: 56
    })
    test: Dataset({
        features: ['Text', 'Label', 'input_ids', 'attention_mask'],
        num_rows: 14
    })
})

In [23]:
tokenized_datasets["train"][0]

{'Text': 'Scientists discover a new species of deep-sea fish in the Mariana Trench.\t',
 'Label': 1,
 'input_ids': [101,
  6529,
  7523,
  1037,
  2047,
  2427,
  1997,
  2784,
  1011,
  2712,
  3869,
  1999,
  1996,
  22097,
  14185,
  1012,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [22]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments("test-trainer",
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=8,
                                  num_train_epochs=10,
                                  learning_rate=2e-5,
                                  weight_decay=0.01,
                                  )

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [21]:
trainer.train()

  0%|          | 0/70 [00:21<?, ?it/s]


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.